# Rest Pagination

Many Rest calls will result in pagination. Dealing with pagination is awkward, so I introduce a function `iter_data`, which helps simplify code logic.

In [7]:
#!/bin/python3

import math
import os
import random
import re
import sys
import requests
import bisect
import operator
from pprint import pp

import requests


def iter_data(
    url_template: str,
    get_total,
    get_data,
):
    page = 1
    total = 1
    while page <= total:
        resp = requests.get(url_template.format(page=page))
        payload = resp.json()
        total = get_total(payload)
        yield from get_data(payload)
        page += 1

In [2]:
# Example: 4+ stars with 10000+ votes
data = iter_data(
    url_template="https://jsonmock.hackerrank.com/api/food_outlets?city=seattle&page={page}",
    get_total=operator.itemgetter('total_pages'),
    get_data=operator.itemgetter('data'),
)
data = filter(
    lambda datum: datum['user_rating']['average_rating'] >= 3.5  and datum['user_rating']['votes'] > 10000,
    data,
)
sorted(data, key=lambda d: d['user_rating']['average_rating'], reverse=True)

[{'city': 'Seattle',
  'name': 'Cafe Juanita',
  'estimated_cost': 160,
  'user_rating': {'average_rating': 4.9, 'votes': 16203},
  'id': 41},
 {'city': 'Seattle',
  'name': "AB's - Absolute Barbecues",
  'estimated_cost': 160,
  'user_rating': {'average_rating': 4.8, 'votes': 11871},
  'id': 424},
 {'city': 'Seattle',
  'name': "Xi'an Noodles",
  'estimated_cost': 140,
  'user_rating': {'average_rating': 4.8, 'votes': 10330},
  'id': 426},
 {'city': 'Seattle',
  'name': 'Truffles',
  'estimated_cost': 90,
  'user_rating': {'average_rating': 4.7, 'votes': 14611},
  'id': 49},
 {'city': 'Seattle',
  'name': 'Toit',
  'estimated_cost': 150,
  'user_rating': {'average_rating': 4.7, 'votes': 14938},
  'id': 423}]

In [3]:
# Example: Cheap food
data = iter_data(
    url_template="https://jsonmock.hackerrank.com/api/food_outlets?city=seattle&page={page}",
    get_total=operator.itemgetter('total_pages'),
    get_data=operator.itemgetter('data'),
)
data = filter(
    lambda datum: datum['estimated_cost'] <= 100,
    data,
)
list(data)

[{'city': 'Seattle',
  'name': 'Truffles',
  'estimated_cost': 90,
  'user_rating': {'average_rating': 4.7, 'votes': 14611},
  'id': 49},
 {'city': 'Seattle',
  'name': 'TBC Sky Lounge',
  'estimated_cost': 10,
  'user_rating': {'average_rating': 4.7, 'votes': 6577},
  'id': 411},
 {'city': 'Seattle',
  'name': 'JuneBaby',
  'estimated_cost': 80,
  'user_rating': {'average_rating': 4.4, 'votes': 1880},
  'id': 413}]

In [11]:
# Example: Most expensive
data = iter_data(
    url_template="https://jsonmock.hackerrank.com/api/food_outlets?city=seattle&page={page}",
    get_total=operator.itemgetter('total_pages'),
    get_data=operator.itemgetter('data'),
)
data = sorted(data, key=operator.itemgetter('estimated_cost'), reverse=True)
data[:10]

[{'city': 'Seattle',
  'name': 'Jetlag',
  'estimated_cost': 1650,
  'user_rating': {'average_rating': 4.1, 'votes': 704},
  'id': 436},
 {'city': 'Seattle',
  'name': 'Baar Union',
  'estimated_cost': 850,
  'user_rating': {'average_rating': 4.7, 'votes': 558},
  'id': 428},
 {'city': 'Seattle',
  'name': 'Biergarten',
  'estimated_cost': 240,
  'user_rating': {'average_rating': 4.7, 'votes': 299},
  'id': 410},
 {'city': 'Seattle',
  'name': 'La Carta de Oaxaca',
  'estimated_cost': 240,
  'user_rating': {'average_rating': 4.7, 'votes': 6829},
  'id': 429},
 {'city': 'Seattle',
  'name': 'XOOX Brewmill',
  'estimated_cost': 200,
  'user_rating': {'average_rating': 4.4, 'votes': 1483},
  'id': 430},
 {'city': 'Seattle',
  'name': 'The Bier Library',
  'estimated_cost': 200,
  'user_rating': {'average_rating': 4.3, 'votes': 476},
  'id': 432},
 {'city': 'Seattle',
  'name': 'Big Pitcher',
  'estimated_cost': 180,
  'user_rating': {'average_rating': 4.7, 'votes': 9011},
  'id': 47},
 {'